<a href="https://colab.research.google.com/github/tantanchen/Stream/blob/main/Data_Cleaning_with_Zephyr_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Open with google colab: https://colab.research.google.com/drive/110ELX40AfeWDZNKBGKXFtpq3HcCcHckM

In [3]:
%%capture
!pip install transformers
!pip install ctransformers[cuda]
!pip install accelerate

In [4]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-beta-GGUF",
                                           model_file="zephyr-7b-beta.Q4_K_M.gguf",
                                           model_type="mistral", gpu_layers = 150)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

(…)2aa9658fa937732cceadc63ac42e/config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

zephyr-7b-beta.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

(…)-beta/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

(…)phyr-7b-beta/resolve/main/tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

(…)r-7b-beta/resolve/main/added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

(…)eta/resolve/main/special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [5]:
eos_token = tokenizer.eos_token
bos_token = tokenizer.bos_token

print(bos_token)
print(eos_token)

<s>
</s>


In [8]:
sys_prompt = "You are a professional data cleaner who only outputs in JSON. \
You will identify all people and their attributes"
example_input = "John and Mary Smith 48 Main St, Somerville MA, 09231"
example_response = """
[
  {
      "first_name": "John",
      "middle_name": "",
      "first_name": "Smith",
      "address": {
        "city": "Somerville",
        "state": "MA",
        "postalCode": "02144",
        "streetAddress": "48 Main St"
      }
  },
  {
      "first_name": "Mary",
      "middle_name": "",
      "first_name": "Smith",
      "address": {
        "city": "Somerville",
        "state": "MA",
        "postalCode": "02144",
        "streetAddress": "48 Main St"
      }
  }
]
"""

input = "Matthew L Keller and Julie Smith 983 Second Ave, Springfield, New York, 19283"

sys_format = "<|system|>\n" + sys_prompt  + "\n <|user|>\n" + example_input
example_format = "<|assistant|>\n<s>\n" + example_response + "</s>"

input_text = sys_format + example_format + "\n<|user|>\n" + input + "\n<|assistant|>\n"
print(input_text)

<|system|>
You are a professional data cleaner who only outputs in JSON. You will identify all people and their attributes
 <|user|>
John and Mary Smith 48 Main St, Somerville MA, 09231<|assistant|>
<s>

[
  {
      "first_name": "John",
      "middle_name": "",
      "first_name": "Smith",
      "address": {
        "city": "Somerville",
        "state": "MA",
        "postalCode": "02144",
        "streetAddress": "48 Main St"
      }
  },
  {
      "first_name": "Mary",
      "middle_name": "",
      "first_name": "Smith",
      "address": {
        "city": "Somerville",
        "state": "MA",
        "postalCode": "02144",
        "streetAddress": "48 Main St"
      }
  }
]
</s>
<|user|>
Matthew L Keller and Julie Smith 983 Second Ave, Springfield, New York, 19283
<|assistant|>



In [9]:
print(model(input_text, temperature=0.01, repetition_penalty =1.1))

<s>
[
  {
    "first_name": "Matthew",
    "middle_name": "L",
    "last_name": "Keller",
    "address": {
      "city": "Springfield",
      "state": "New York",
      "postalCode": "19283",
      "streetAddress": "983 Second Ave"
    }
  },
  {
    "first_name": "Julie",
    "middle_name": "",
    "last_name": "Smith",
    "address": {
      "city": "Springfield",
      "state": "New York",
      "postalCode": "19283",
      "streetAddress": "983 Second Ave"
    }
  }
]
</s>
